### Imports

In [ ]:
from tensorflow.keras.models import load_model
import pandas as pd
import pathlib
import tensorflow as tf
import os
import tensorflow.keras.applications     as     applications
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
import seaborn as sns

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys
sys.path.append('../utilities/')
from utilities import Timer, show_image, DreamImage, DeepDream, class_names, ProgressBar, plot_confusion_matrix

### Setup

In [2]:
sns.set()

AUTOTUNE = tf.data.experimental.AUTOTUNE

width = height = 224
batch_size = 32

### Load the Model

In [ ]:
model_name = 'vgg19-INet-down2-b'
base_model = load_model(f'../classification/logs/models/{model_name}.hdf5')

# Remove dropout
model = Sequential()
for layer in base_model.layers:
    if isinstance(layer, Dropout):
        pass
    else:
        model.add(layer)

### Load the Data in Special Form

In [ ]:
# Load the list of file names in the form of data_dir/class_name/file_name.jpg
data_subdir = 'test'
data_dir = pathlib.Path('../dataset/images/') / pathlib.Path(data_subdir)
list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'))

dataset_size = len(list(list_ds))

In [ ]:
# Function to get the label
def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2] == class_names

# Function to get the image
def decode_img(file_path):
    # Load the image
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)

    # Resize
    return tf.image.resize(img, [width, height])

def preprocess_img(img, label):
    return applications.vgg19.preprocess_input(img), label

In [ ]:
# Combine both processes
def process_path(file_path):
    return decode_img(file_path), get_label(file_path)

In [ ]:
def classify_image(img_batch, label_batch):
    return model(img_batch), label_batch

In [ ]:
# Create the dataset
ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
ds = ds.map(preprocess_img)
ds = ds.batch(batch_size).prefetch(buffer_size = AUTOTUNE)
ds = ds.map(classify_image)
ds = ds.cache()

### Do Predictions on the Dataset

In [ ]:
bar = ProgressBar(dataset_size // batch_size + 1)

In [ ]:
class_batches = []
label_batches = []
bar.start()
iterations = 0
for class_batch, label_batch in ds:
    class_batches.append(class_batch)
    label_batches.append(label_batch)
    iterations+=1
    bar.update(iterations)

### Extract Predictions into a Convenient Numpy Array

In [ ]:
# Convert to a single array
def concat_batches(batch_list):
    batch_list = [tensor.numpy() for tensor in batch_list]
    return np.concatenate(batch_list)

y_predict_proba = concat_batches(class_batches)
y_true          = concat_batches(label_batches)

In [ ]:
# Get Hard Predictions
y_pred = []
for a in y_predict_proba:
    out = np.zeros(len(a))
    out[a.argmax()] = 1
    out = np.expand_dims(out, 0)
    y_pred.append(out)
y_pred = np.concatenate(y_pred)

In [ ]:
# Get Predictions and True Values as Labels
y_pred_label = []
for a in y_predict_proba:
    out = class_names[a.argmax()]
    y_pred_label.append(out)

y_true_label = []
for a in y_true:
    out = class_names[a.argmax()]
    y_true_label.append(out)

### Examine Predictions

In [ ]:
print(classification_report(y_true_label, y_pred_label))

In [ ]:
np.set_printoptions(threshold=np.inf)

cm = confusion_matrix(y_true_label, y_pred_label)
cm

In [ ]:
fig, ax = plt.subplots(figsize = (20, 20))
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=class_names)
disp.plot(include_values=True,
                 cmap=sns.cubehelix_palette(light=1, as_cmap=True), ax=ax, xticks_rotation='vertical')
plt.show()